In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional 
import numpy as np
INITIAL_BALANCE = 5000

In [2]:
df_xau = pd.read_csv("Files/backtest_formateado_XAU_M5.csv")
df = pd.read_excel(r"Files/the5ers_bots.xlsx",sheet_name="All")

#Rellenando na con ceros 
df.fillna(0,inplace=True)


#Colocando comment en operaciones manuales
df.loc[df["Comment"]==0,"Comment"] = "Manual"

#Resetando indice
df.reset_index(inplace=True,drop=True)

#Rename a dos columnas que se llamaban iguales
df.rename(columns={"Price":"Price Entry", "Price.1":"Price Exit"},inplace=True)
df.rename(columns={"Time":"Time Entry","Time.1":"Time Exit"},inplace=True)

#Nueva columna con el balance 
# df["Balance"] = INITIAL_BALANCE + (df["Profit"]+df["Commission"]+df["Swap"]).cumsum() 



# Funciones

In [3]:
def get_bots_by_comment():
    return df["Comment"].unique()

def handle_errors_not_string_and_not_in_df(x):
     #Verificicar que sea none para no entrar en el if y que sea una string 
    if x is not None:
        if  not isinstance(x,str) :
            raise TypeError(f"{x} no es una string")
    
        #Conseguir los bots por comentario del df y verificar si bot se encuentra
        if x not in get_bots_by_comment():
            raise KeyError(f"{x} no está en el dataframe")    

In [4]:
def profit_factor_v2(data,by_bot=True):
    
    #Profit factor por bot
    if by_bot:
        x = data.groupby("Comment")        

        return x.apply(profit_factor_v2,by_bot=False,include_groups=False) #type: ignore


    total_profit = data[data["Profit"]>=0]["Profit"].sum()
    total_loss = data[data["Profit"]< 0]["Profit"].sum()

    #Division por 0
    if total_loss==0:
        return float('inf')
    

    return round(abs(total_profit/total_loss),2)
    

In [5]:
def win_rate(data:pd.DataFrame,by_bot = False,BE = True):
    """
    Calcula los ratios para los bots o para el total de la cuenta
    Se puede desactivar el calculo con BE y hacerlo normal

    Nota: Se considera BE cuando el profit fue menor que el 0.15% del balance actual
    """

    #data es un dataframe?
    if not isinstance(data,pd.DataFrame):
        raise TypeError("data is not a Dataframe")
    
    #Profit y balance son columnas necesarias para el calculo, están?
    if "Profit" not in data.columns or "Balance" not in data.columns:
        raise KeyError("Columns 'Profit' or 'Balance' is not in your dataframe")

  

    #Calculo
    total = data["Profit"].count()

    percent = lambda x: round(x*100/total,2)  

    win = ((data["Profit"] > 0) & (data["Profit"] > data["Balance"]*0.0015)).sum() 
    win_percent = percent(win)

    losses = (data["Profit"] < 0 ).sum()
    losses_percent = percent(losses)
    
    #Diccionario base
    dicc =  {"Win":[f"{win_percent}%",win],
            "Loss":[f"{losses_percent}%",losses]}

    #Queres los breakeven?
    if BE:
        breakeven =  ((data["Profit"] < data["Balance"]*0.0015) & (data["Profit"] > 0)).sum()
        breakeven_percent = percent(breakeven)

        dicc["BE"] = [f"{breakeven_percent}%",breakeven]
    
    #Añado el total
    
    dicc["Total"] = ["100%", total]


    #Para cada bot
    if by_bot:
        x = data.groupby("Comment")[["Profit","Balance"]]
        return x.apply(win_rate,BE=BE)
    

    return pd.DataFrame(dicc,index=["Percent","n"])

In [6]:
def balance_in_R(data:pd.DataFrame,RISK:float = 0.01):
    
    # * Formula: Balance_n = Balance_0 * PRODUCTORIO(1 + R_n* RISK) 
    
    #Limpio "R(usd)" que no me interesa
    if "R(usd)" in data.columns:
        data = data.drop(columns="R(usd)")

    #Interior del Productorio
    data["growth"] = 1+ data["R-multiple"]*RISK
    
    #Toda la Formula
    data["Balance_with_R"] = (INITIAL_BALANCE *data["growth"].cumprod()).round(2)
    
    
    return data[data.columns[-6:]]

# Cosas Varias

In [ ]:
from logic import BotAnalyzer 

fiveers = BotAnalyzer(df)
fiveers.add_R()

fiveers_df = fiveers.df

# fiveers_df.groupby("Comment").apply(BotAnalyzer.expectancy_R,include_groups=False)

fiveers_df.groupby("Comment").apply(BotAnalyzer.profit_factor,include_groups=False) #type:ignore
fiveers_df.groupby("Comment").apply(BotAnalyzer.win_rate,include_groups=False) #type:ignore

Win    Loss Total
Comment                                                 
Manual                     Percent    0.0%  100.0%  100%
                           n             0       2     2
NQ_WF_Strategy_4_10_26_1_1 Percent  100.0%    0.0%  100%
                           n             1       0     1
Porfolio 1.2.3 with SL     Percent  66.67%  33.33%  100%
                           n             2       1     3
SP_Strategy_5_8_16         Percent    0.0%  100.0%  100%
                           n             0       2     2
XAU_M5                     Percent  66.67%  33.33%  100%
                           n             8       4    12